<a href="https://colab.research.google.com/github/DiegoLangreo7/EII-RI/blob/main/Ejercicio3_RI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ej. 3. Expansión de consultas**

En este nuevo ejercicio, se nos pide que, utilizando la mejor configuración determinada en el segundo ejercicio, implementar un sistema de expansión de consultas empleando la técnica del signed root log likelihood ratio test.

Volvemos a repetir el proceso inicial, para tener la coleccion operativa, es decir, ya indexada:


In [ ]:
# Instalamos el paquete bm25s
!pip install bm25s[full]

# Importamos las librerías necesarias
import bm25s
import Stemmer
import json
from collections import Counter

# Descargamos la colección y las consultas y las query
!gdown 19pzNFYIch8rj9d3kyq-171V8vRa_wLBC
!unzip -o trec-covid-RI.zip

# Parseamos la colección
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    corpus_content = []
    for line in f:
        corpus_content.append(json.loads(line))

# Indexamos la colección
corpus_verbatim = list()
corpus_plaintext = list()
for entry in corpus_content:
  document = {"id": entry["_id"], "title": entry["title"].lower(),
  "text": entry["text"].lower()}
  corpus_verbatim.append(document)
  corpus_plaintext.append(entry["text"].lower())

Una vez ya tenemos nuestra coleccion, utilizamos una funcion proporcionada por el profesor de practicas para facilitarnos la tarea. Con ella nos permite calcular las frecuencias de términos para nuestro corpus.

In [ ]:
def compute_term_frequencies_from_corpus_tokenized(corpus_tokenized):
    from collections import Counter

    tmp = dict()

    for document in corpus_tokenized[0]:
        freqs = dict(Counter(document))
        for token, freq in freqs.items():
            try:
                tmp[token] += freq
            except:
                tmp[token] = freq

    inverted_vocab = {corpus_tokenized[1][key]: key for key in corpus_tokenized[1].keys()}

    total_freqs = dict()

    for key, freq in dict(tmp).items():
        term = inverted_vocab[key]
        total_freqs[term] = freq

    return total_freqs

Una vez tengamos la coleccion inicial indexada, y la funcion declarada, dejamos ya calculados los terminos para la coleccion general:

In [ ]:
bm25_flavor = "robertson"
idf_flavor = "robertson"

initial_corpus_tokenized = bm25s.tokenize(corpus_plaintext, stopwords=None, stemmer=Stemmer.Stemmer("english"))
initial_compute_term_frequencies = compute_term_frequencies_from_corpus_tokenized(initial_corpus_tokenized)
print(initial_compute_term_frequencies)

Ahora, dejaremos las consultas cargadas desde nuestro fichero de consultas, ademas de dejar preparado el retriever para su posterior uso.

In [ ]:
#queries cargadas
queries = []

with open("queries.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        queries.append(json.loads(line))

retriever = bm25s.BM25(corpus=corpus_verbatim,method=bm25_flavor,idf_method=idf_flavor)
retriever.index(initial_corpus_tokenized, show_progress=True)

Ahora, para cada consulta sera necesario mediante el algoritmo de LLRs, expandirla añadiendo a cada uno los terminos mas populares contrastados con los terminos populares de la coleccion en general. Para ello creamos el metodo de expandir consultar, para que para cada query reciba su expansion y contrastar esta mejora con la respuesta run sin ella.

In [ ]:
# Devuelve el texto plano de los n documentos mas relevantes
def get_n_valuable_documents_for_query(query, retriever, n):
    # Convierte el texto de la consulta a minúsculas
    query_string = query["text"].lower()

    # Tokeniza la consulta
    query_tokenized = bm25s.tokenize(
        query_string,
        stopwords=None,
        stemmer=Stemmer.Stemmer("english"),
        show_progress=False
    )

    # Recupera los documentos relevantes usando el índice BM25
    results = retriever.retrieve(
        query_tokenized,
        corpus=retriever.corpus,
        k=n,
        return_as="tuple",
        show_progress=False
    )

    # Itera sobre los documentos recuperados y agrega su texto a text_content
    text_content = ""
    for document in results.documents[0]:
        text_content += document['text'] + " "

    return text_content


# Devuelve los terminos con mas frecuencia para un texto plano (el de la query)
def get_term_frequency_for_query(query_text):
    queried_corpus_tokenized = bm25s.tokenize(query_text, stopwords=None, stemmer=Stemmer.Stemmer("english"))
    return compute_term_frequencies_from_corpus_tokenized(queried_corpus_tokenized)


# Compara las frecuencias de los terminos de toda la coleccion, con la los archivos de la query
def comparate_frequencies(dictA, dictB, m):
    result = LogLikelihood.compare_frequencies(dictA, dictB, max_return=m, threshold=0)
    return result


# Expande la query para implementar los terminos resutantes del algoritmo explicado
def query_expansion(query, retriever, n, m):
    text = get_n_valuable_documents_for_query(query, retriever, n)
    term_frequencies = get_term_frequency_for_query(text)
    tf_terms = comparate_frequencies(initial_compute_term_frequencies, term_frequencies, m)

    # Accede al atributo item de cada ScoredItem
    tf_terms_items = [item.item for item in tf_terms]

    original_terms = query["text"].split()

    # Extiende la consulta con los términos relevantes
    the_extended_one = original_terms[:]
    for term in tf_terms_items:
        if term not in the_extended_one:
            the_extended_one.append(term)

    return the_extended_one

Para calcular los LLR con signo, se nos proporciona un codigo de ejemplo en lenguaje java, para ello traducimos este texto a python para su uso en este programa:

In [ ]:
import math
from heapq import heappush, heappop

class LogLikelihood:
    @staticmethod
    def entropy(*elements):
        total_sum = sum(elements)
        result = sum(LogLikelihood.x_log_x(e) for e in elements)
        return LogLikelihood.x_log_x(total_sum) - result

    @staticmethod
    def x_log_x(x):
        return 0.0 if x == 0 else x * math.log(x)

    @staticmethod
    def log_likelihood_ratio(k11, k12, k21, k22):
        if any(k < 0 for k in [k11, k12, k21, k22]):
            raise ValueError("Counts must be non-negative.")

        row_entropy = LogLikelihood.entropy(k11 + k12, k21 + k22)
        column_entropy = LogLikelihood.entropy(k11 + k21, k12 + k22)
        matrix_entropy = LogLikelihood.entropy(k11, k12, k21, k22)

        if row_entropy + column_entropy < matrix_entropy:
            return 0.0

        return 2.0 * (row_entropy + column_entropy - matrix_entropy)

    @staticmethod
    def root_log_likelihood_ratio(k11, k12, k21, k22):
        llr = LogLikelihood.log_likelihood_ratio(k11, k12, k21, k22)
        sqrt_llr = math.sqrt(llr)
        if k11 / (k11 + k12) < k21 / (k21 + k22):
            sqrt_llr = -sqrt_llr
        return sqrt_llr

    @staticmethod
    def compare_frequencies(a, b, max_return, threshold):
        total_a = sum(a.values())
        total_b = sum(b.values())

        best = []

        for item in a:
            LogLikelihood._compare_and_add(a, b, max_return, threshold, total_a, total_b, best, item)

        if threshold < 0:
            for item in b:
                if item not in a:
                    LogLikelihood._compare_and_add(a, b, max_return, threshold, total_a, total_b, best, item)

        return sorted(best, key=lambda x: -x.score)

    @staticmethod
    def _compare_and_add(a, b, max_return, threshold, total_a, total_b, best, item):
        k_a = a.get(item, 0)
        k_b = b.get(item, 0)
        score = LogLikelihood.root_log_likelihood_ratio(k_a, total_a - k_a, k_b, total_b - k_b)

        if score >= threshold:
            heappush(best, ScoredItem(item, score))
            if len(best) > max_return:
                heappop(best)

class ScoredItem:
    def __init__(self, item, score):
        self.item = item
        self.score = score

    def __lt__(self, other):
        return self.score < other.score

Teniendo todo lo anterior, podremos expandir nuestras queries:

In [ ]:
# Expande la lista de queries para unos parametros dados
def get_expanded_queries(queries, retriever, n, m):
    expanded_queries = []
    for query in queries:
        expanded_query = query_expansion(query, retriever, n, m)
        expanded_queries.append({
            "_id": query["_id"],
            "text": " ".join(expanded_query)
        })
    return expanded_queries

Una vez tenemos las consultas ampliadas, haremos un run con el metodo ya usado en el ejercicio anterior con el fin de volver a enviar al índice BM25S para obtener la lista definitiva de resultados, luego usaremos estos resultados para compararlos con el resultado sin las query expandidas, con otra vez, un metodo del anterior ejercicio.

In [ ]:
def submit_queries_and_get_run(queries, retriever, max_results=100):
    # Inicializa un diccionario para almacenar los resultados de cada consulta
    run = {}

    # Itera sobre cada consulta en la lista de consultas
    for query in queries:
        # Obtiene el ID único de la consulta
        query_id = query["_id"]

        # Convierte el texto de la consulta a minúsculas
        query_string = query["text"].lower()

        # Tokeniza la consulta utilizando las stopwords y el stemmer proporcionados
        query_tokenized = bm25s.tokenize(
            query_string,
            stopwords=None,
            stemmer=Stemmer.Stemmer("english"),
            show_progress=False
        )

        # Recupera los documentos relevantes usando el índice BM25
        results = retriever.retrieve(
            query_tokenized,
            corpus=retriever.corpus,
            k=max_results,
            return_as="tuple",
            show_progress=False
        )

        # Obtiene los documentos recuperados y sus scores de relevancia
        returned_documents = results.documents[0]

        # Inicializa una lista para almacenar los IDs de los documentos recuperados
        returned_ids = []
        for i in range(len(returned_documents)):
            # Agrega el ID de cada documento recuperado a la lista
            returned_ids.append(str(returned_documents[i]["id"]))

        # Asocia la lista de IDs recuperados con el ID de la consulta en el diccionario
        run[query_id] = returned_ids

    # Devuelve el diccionario con los resultados de todas las consultas
    return run


# Función adaptada para calcular precision, recall y F1

def compute_precision_recall_f1(run, relevance_judgements):
    precision_values = []
    recall_values = []
    f1_values = []

    global_retrieved = 0
    global_relevant = 0
    global_retrieved_and_relevant = 0

    for query_id in run.keys():
        retrieved_results = run[query_id]
        relevant_results = relevance_judgements.get(query_id, [])
        relevant_and_retrieved = set(retrieved_results) & set(relevant_results)

        global_retrieved += len(retrieved_results)
        global_relevant += len(relevant_results)
        global_retrieved_and_relevant += len(relevant_and_retrieved)

        precision = len(relevant_and_retrieved) / len(retrieved_results) if len(retrieved_results) > 0 else 0
        recall = len(relevant_and_retrieved) / len(relevant_results) if len(relevant_results) > 0 else 0

        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_values.append(f1)

        precision_values.append(precision)
        recall_values.append(recall)

    macro_average_precision = sum(precision_values) / len(precision_values) if precision_values else 0
    macro_average_recall = sum(recall_values) / len(recall_values) if recall_values else 0
    macro_average_f1 = sum(f1_values) / len(f1_values) if f1_values else 0

    micro_average_precision = global_retrieved_and_relevant / global_retrieved if global_retrieved > 0 else 0
    micro_average_recall = global_retrieved_and_relevant / global_relevant if global_relevant > 0 else 0
    micro_average_f1 = (2 * (micro_average_precision * micro_average_recall) /
                        (micro_average_precision + micro_average_recall)) if (micro_average_precision + micro_average_recall) > 0 else 0

    return {
        "macro_precision": macro_average_precision,
        "macro_recall": macro_average_recall,
        "macro_f1": macro_average_f1,
        "micro_precision": micro_average_precision,
        "micro_recall": micro_average_recall,
        "micro_f1": micro_average_f1
    }


# Cargar juicios de relevancia desde qrels.tsv
relevance_judgements = {}
with open("qrels.tsv", "r", encoding="utf-8") as f:
    next(f)  # Saltar la cabecera
    for line in f:
        query_id, corpus_id, score = line.strip().split("\t")
        if int(score) > 0:  # Considerar solo documentos relevantes (score > 0)
            if query_id not in relevance_judgements:
                relevance_judgements[query_id] = []
            relevance_judgements[query_id].append(corpus_id)


Ahora por cada conjunto de queries, obtenemos el run y comparamos



In [ ]:
run_original = submit_queries_and_get_run(queries, retriever, max_results=100)

metrics = compute_precision_recall_f1(run_original, relevance_judgements)
print("Resultados obtenidos de la ejecucion sin las consultas expandidas:\n")
print(f"Macro-averaged Precision: {metrics['macro_precision']:.3f}")
print(f"Macro-averaged Recall: {metrics['macro_recall']:.3f}")
print(f"Macro-averaged F1: {metrics['macro_f1']:.3f}\n")
print(f"Micro-averaged Precision: {metrics['micro_precision']:.3f}")
print(f"Micro-averaged Recall: {metrics['micro_recall']:.3f}")
print(f"Micro-averaged F1: {metrics['micro_f1']:.3f}\n")

expanded_queries = get_expanded_queries(queries, retriever, 100, 5)
run_original = submit_queries_and_get_run(expanded_queries, retriever, max_results=100)

metrics = compute_precision_recall_f1(run_original, relevance_judgements)
print("Resultados obtenidos de la ejecucion con las consultas expandidas:\n")
print(f"Macro-averaged Precision: {metrics['macro_precision']:.3f}")
print(f"Macro-averaged Recall: {metrics['macro_recall']:.3f}")
print(f"Macro-averaged F1: {metrics['macro_f1']:.3f}\n")
print(f"Micro-averaged Precision: {metrics['micro_precision']:.3f}")
print(f"Micro-averaged Recall: {metrics['micro_recall']:.3f}")
print(f"Micro-averaged F1: {metrics['micro_f1']:.3f}\n")


Aqui podemos contrastar resultados, en donde dependiendo del n y el m es mas rentable la expansion o no.